In [53]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SequentialFeatureSelector, RFE

In [54]:
# Collect file paths
files = glob.glob('data/*/*/*.txt')

# Filter out annotations.txt files
files = [x for x in files if "annotations" not in x] 

# Get miRNA IDs as columns
col_ids = pd.read_csv(files[0], delimiter= '	', header=0)[['miRNA_ID']].T[:].values[0]

num_files = len(files)

print(num_files)
print(col_ids)

2895
['hsa-let-7a-1' 'hsa-let-7a-2' 'hsa-let-7a-3' ... 'hsa-mir-98'
 'hsa-mir-99a' 'hsa-mir-99b']


In [55]:
df_raw = pd.DataFrame(columns = [])

# Load df_raw
for i, path in enumerate(files):
    # Read in data
    df = pd.read_csv(path, delimiter= '	', header=0)
    
    # Isolate features
    df['Feature_value'] = df['reads_per_million_miRNA_mapped']
    df = df[['Feature_value']]
    
    # Build row of features + target label
    cancer_type = str(Path(path).parent.parent)
    index_of_label = str(cancer_type).index("/") + 1
    cancer_type = cancer_type[index_of_label:len(str(cancer_type))]
    row = pd.DataFrame(df.T.assign(Target = np.repeat(cancer_type, 1)))
    
    # Append row to df_raw
    df_raw = df_raw.append(row, ignore_index = True)

In [56]:
df_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,1872,1873,1874,1875,1876,1877,1878,1879,1880,Target
0,17225.641202,17168.697999,17165.904559,37132.340301,4217.020216,410.420821,1099.541016,4846.188888,4989.513855,655.169154,...,1.933920,0.0,3.867840,5.586880,0.0,31.157602,47.703362,1462.043595,37550.067043,Breast_Invasive_Carcinoma
1,9675.101346,9620.924588,9710.866472,11593.826262,1817.884192,334.477994,1189.031704,4199.545275,4264.938315,511.504784,...,5.925583,0.0,1.269768,1.058140,0.0,26.136053,44.547686,668.744359,13635.189582,Breast_Invasive_Carcinoma
2,9947.288063,10160.137808,10204.137755,9738.288314,1366.198361,243.649708,813.999023,735.899117,752.399097,195.799765,...,0.549999,0.0,0.000000,3.849995,0.0,26.399968,24.199971,228.249726,33884.359339,Breast_Invasive_Carcinoma
3,18022.771624,18041.827151,18067.323984,17540.076324,3884.643741,230.679238,2396.299685,8376.112098,8537.815694,681.436400,...,3.891622,0.0,1.476132,2.549683,0.0,26.033608,49.785921,1360.323117,17850.466713,Breast_Invasive_Carcinoma
4,4686.419964,4688.795641,4698.100379,2814.682994,323.884043,234.697147,1773.938243,4904.982301,5131.364576,262.215412,...,0.296960,0.0,0.098987,0.296960,0.0,123.535234,43.554089,89.978788,28231.760414,Breast_Invasive_Carcinoma


In [57]:
# Promote first row to headers
data_renamed = pd.DataFrame(columns = [])

for i, col in enumerate(col_ids):
    data_renamed[col] = df_raw[i]
    
data_renamed['Target'] = df_raw['Target']
data_renamed.head()

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b,Target
0,17225.641202,17168.697999,17165.904559,37132.340301,4217.020216,410.420821,1099.541016,4846.188888,4989.513855,655.169154,...,1.933920,0.0,3.867840,5.586880,0.0,31.157602,47.703362,1462.043595,37550.067043,Breast_Invasive_Carcinoma
1,9675.101346,9620.924588,9710.866472,11593.826262,1817.884192,334.477994,1189.031704,4199.545275,4264.938315,511.504784,...,5.925583,0.0,1.269768,1.058140,0.0,26.136053,44.547686,668.744359,13635.189582,Breast_Invasive_Carcinoma
2,9947.288063,10160.137808,10204.137755,9738.288314,1366.198361,243.649708,813.999023,735.899117,752.399097,195.799765,...,0.549999,0.0,0.000000,3.849995,0.0,26.399968,24.199971,228.249726,33884.359339,Breast_Invasive_Carcinoma
3,18022.771624,18041.827151,18067.323984,17540.076324,3884.643741,230.679238,2396.299685,8376.112098,8537.815694,681.436400,...,3.891622,0.0,1.476132,2.549683,0.0,26.033608,49.785921,1360.323117,17850.466713,Breast_Invasive_Carcinoma
4,4686.419964,4688.795641,4698.100379,2814.682994,323.884043,234.697147,1773.938243,4904.982301,5131.364576,262.215412,...,0.296960,0.0,0.098987,0.296960,0.0,123.535234,43.554089,89.978788,28231.760414,Breast_Invasive_Carcinoma


In [58]:
data_shuffled = shuffle(data_renamed)

# y: Targets
y = data_shuffled['Target']

# X: Features
X = data_shuffled[col_ids]

nrow = len(data_shuffled.index)

# Find split lengths
train_len = round(nrow * 0.6)
tv_len = round(nrow * 0.2)
t_end = train_len+tv_len

In [59]:
# Train-test-validate split
X_train, y_train = X.iloc[:train_len], y.iloc[:train_len]
X_test, y_test = X.iloc[train_len:t_end], y.iloc[train_len:t_end]
X_val, y_val = X.iloc[t_end:], y.iloc[t_end:]

X_val.head()

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
755,12284.278004,12258.290856,12172.686131,23367.796878,1126.619324,639.742452,1281.777888,4631.062744,4738.068650,651.207371,...,0.0,5.350295,0.0,0.000000,10.700591,0.0,6.114623,46.624002,307.259816,67763.782934
2524,4469.975700,4333.172459,4457.940225,22085.098035,261.972189,629.455383,428.061755,1096.030660,1106.060223,266.385197,...,0.0,2.005913,0.0,162.478923,3.209460,0.0,32.896967,62.183291,89.463703,20922.872258
985,7395.355646,7299.342931,7324.534386,41750.321310,617.903615,1089.411604,729.601576,1463.480946,1475.363708,1017.164412,...,0.0,15.209935,0.0,0.950621,7.604968,0.0,100.765820,39.926080,176.815496,34490.429148
1748,4934.741503,4986.762076,5034.629746,5249.706401,657.688673,462.502238,604.793805,2308.795433,2368.028943,427.530424,...,0.0,6.994363,0.0,2.622886,0.437148,0.0,28.851746,33.878945,156.280293,16558.716692
2740,4701.481252,4669.800552,4686.784020,13659.607532,734.535001,565.680134,506.564600,1157.488681,1195.701484,489.907737,...,0.0,3.919262,0.0,0.326605,9.471550,0.0,42.132065,30.700885,206.741065,19140.042086


Below is an old scaler that was manually coded and had some bugs. It has been commented out and replaced with sklearn MinMaxScaler below the commented out code.

In [60]:
#Old scaler manually coded had some bugs replaced with sklearn MinMaxScaler below

# def scale_col(vector):
#     minima = np.min(vector, axis=0)
#     maxima = np.max(vector, axis=0)
    
#     return (vector - minima) / (maxima - minima)

In [61]:
#Old scaler manually coded had some bugs replaced with sklearn MinMaxScaler below

# data_renamed = pd.DataFrame(columns = [])

# for i, col in enumerate(col_ids):
#     X_train[col] = scale_col(X_train[col])
#     X_test[col] = scale_col(X_test[col])
    
# y_train = scale_col(y_train)
# y_test = scale_col(y_test)

In [62]:
#scaling xtrain using sklearn MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
Xtrain_scaled = pd.DataFrame(scaler.transform(X_train))

#next scale X_test
Xtest_scaled = pd.DataFrame(scaler.transform(X_test))
Xtest_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880
0,0.083692,0.084818,0.083450,0.088541,0.040435,0.154921,0.054070,0.055401,0.055759,0.182758,...,0.0,0.043772,0.0,0.000711,0.011893,0.0,0.041531,0.114160,0.066028,0.087799
1,0.169404,0.170056,0.168440,0.050478,0.024947,0.051899,0.112727,0.218483,0.221065,0.226207,...,0.0,0.013628,0.0,0.000856,0.012887,0.0,0.067763,0.166261,0.017222,0.044037
2,0.083547,0.086542,0.088501,0.203709,0.022542,0.057903,0.080057,0.018918,0.019160,0.090384,...,0.0,0.011732,0.0,0.000393,0.000000,0.0,0.161179,0.186171,0.023632,0.090696
3,0.112684,0.112710,0.112116,0.059707,0.031683,0.099227,0.087754,0.175768,0.175685,0.085282,...,0.0,0.018249,0.0,0.000245,0.034769,0.0,0.006997,0.126977,0.027438,0.061797
4,0.329923,0.331825,0.331639,0.297639,0.021523,0.102888,0.213356,0.288474,0.291291,0.282395,...,0.0,0.010921,0.0,0.000488,0.010199,0.0,0.288433,0.136024,0.025133,0.128082


Below section commented out because svm.SVC can only be run once per all-cell-run for some reason

In [63]:
# #SVM hyperparameter tuning
# svm = svm.SVC(decision_function_shape = 'ovr')
# parameters = {'kernel':('linear', 'rbf', 'sigmoid'), 'C':(3,4,5,6)} #experiment w kernel & regularization w gridsearch
# clf = GridSearchCV(svm, parameters)#, random_state=0)
# search = clf.fit(Xtrain_scaled, y_train)
# search.best_params_   #GridSearch suggests that the best params are kernel=linear and c=5

In [64]:
#SVM implementation - 
# svmClf = svm.SVC(kernel = 'linear', C = 5, decision_function_shape = 'ovr')
# svmClf.fit(Xtrain_scaled, y_train)
# svmClf.score(Xtest_scaled, y_test)
#check scores all at once to avoid leaking knowledge of validation set from reshuffle

In [65]:
#RandomForest hyperparameter tuning
# rf = RandomForestClassifier() #experiment with depth, max_features, and n_estimators
# parameters = {'n_estimators':(65,68,70), 'max_depth':(9,10,11), 'max_features':('auto', 'log2')} #auto=43, log2=10
# clf = RandomizedSearchCV(rf, parameters, random_state=0)
# search = clf.fit(Xtrain_scaled, y_train)
# search.best_params_
#RandomizedSearch suggesting n_estimators = 70, max_features = auto, max_depth = 10

In [66]:
#RandomForest implementation
# rfClf = RandomForestClassifier(n_estimators=70, max_features='auto', max_depth=10)
# rfClf.fit(Xtrain_scaled, y_train)
# rfClf.score(Xtest_scaled, y_test)
#check scores all at once to avoid leaking knowledge of validation set from reshuffle

In [67]:
#AdaBoost hyperparameter tuning
# ab = AdaBoostClassifier(random_state=0) #experiment with learning_rate and n_estimators
# parameters2 = {'n_estimators':(45,50,55), 'learning_rate':(0.5,0.55,0.6)}
# clf2 = GridSearchCV(ab, parameters2)#, random_state=0)
# search2 = clf2.fit(Xtrain_scaled, y_train)
# search2.best_params_
#gridsearch gives learning_rate = 0.6 and n_estimators = 50

In [68]:
#AdaBoost implementation
# abClf = AdaBoostClassifier(n_estimators = 50, learning_rate = 0.6)
# abClf.fit(Xtrain_scaled, y_train)
# abClf.score(Xtest_scaled, y_test)
#check scores all at once to avoid leaking knowledge of validation set from reshuffle

In [69]:
#score of classifier that always predicts most common target value
counts = y.value_counts() #1096 BIC out of 2895 total
baseline_acc = 1096/2895
baseline_acc #37.858% accuracy if we always assign the most common target label

0.37858376511226255

Noteworthy: SVM and RandomForest are outperforming AdaBoost. SVM has a regularization term and RandomForest only looks at a few features at a time whereas AdaBoost is looking at all features at all times. I will perform some sort of feature selection (either sfs or rfe) and run all three models on a dataset using only the most significant 30% of the features (this decision informed by the findings of the paper that at 30% feature use no sensitivity is lost)

In [75]:
#defining the number of features we'd like to keep
cutoff = round(len(X.columns)*0.3)

#selecting features for svm
svmClf2 = svm.SVC(kernel = 'linear', C = 5, decision_function_shape = 'ovr')
sfs1 = RFE(svmClf2, n_features_to_select=cutoff, step=20)
svmX = pd.DataFrame(sfs1.fit_transform(Xtrain_scaled, y_train))
svmX.head()

,0,1,2,3,4,5,6,7,8,9,...,554,555,556,557,558,559,560,561,562,563
0,0.131750,0.132406,0.132887,0.154800,0.036081,0.175877,0.081992,0.132159,0.048501,0.096373,...,0.075109,0.003262,0.005162,0.275297,0.044281,0.022960,0.047453,0.128861,0.034360,0.064539
1,0.062076,0.063212,0.063182,0.100384,0.027359,0.066596,0.158198,0.211972,0.114630,0.155920,...,0.090073,0.003274,0.006216,0.033155,0.041341,0.027651,0.083754,0.110268,0.028488,0.081246
2,0.258022,0.257324,0.257428,0.187498,0.025800,0.258924,0.218783,0.241181,0.122059,0.042769,...,0.052731,0.000668,0.000000,0.101472,0.001265,0.035261,0.039203,0.121011,0.012075,0.086572
3,0.141532,0.143158,0.141974,0.239877,0.032364,0.055802,0.101570,0.101494,0.038567,0.158063,...,0.039919,0.001409,0.001115,0.261608,0.000445,0.002479,0.065025,0.081740,0.048924,0.072693
4,0.071343,0.073680,0.073416,0.144401,0.017497,0.100567,0.109249,0.159125,0.035075,0.071049,...,0.178020,0.006797,0.016131,0.114714,0.002146,0.065773,0.132958,0.320964,0.022079,0.226035


In [76]:
#selecting features for RandomForest
rfClf2 = RandomForestClassifier(n_estimators=70, max_features='auto', max_depth=10)
sfs2 = RFE(rfClf2, n_features_to_select=cutoff, step=20)
rfX = pd.DataFrame(sfs2.fit_transform(Xtrain_scaled, y_train))
rfX.head()

,0,1,2,3,4,5,6,7,8,9,...,554,555,556,557,558,559,560,561,562,563
0,0.131750,0.132406,0.132887,0.154800,0.036081,0.081126,0.175877,0.076741,0.077035,0.081992,...,0.005162,0.184598,0.000000,0.025608,0.044281,0.022960,0.047453,0.128861,0.034360,0.064539
1,0.062076,0.063212,0.063182,0.100384,0.027359,0.094553,0.066596,0.035190,0.035386,0.158198,...,0.006216,0.005053,0.060680,0.032383,0.041341,0.027651,0.083754,0.110268,0.028488,0.081246
2,0.258022,0.257324,0.257428,0.187498,0.025800,0.240699,0.258924,0.223620,0.220599,0.218783,...,0.000000,0.008591,0.042856,0.033036,0.001265,0.035261,0.039203,0.121011,0.012075,0.086572
3,0.141532,0.143158,0.141974,0.239877,0.032364,0.049563,0.055802,0.048118,0.050000,0.101570,...,0.001115,0.007249,0.040178,0.014379,0.000445,0.002479,0.065025,0.081740,0.048924,0.072693
4,0.071343,0.073680,0.073416,0.144401,0.017497,0.073406,0.100567,0.024778,0.022817,0.109249,...,0.016131,0.122373,0.096898,0.010671,0.002146,0.065773,0.132958,0.320964,0.022079,0.226035


In [ ]:
#selecting features for AdaBoost
abClf2 = AdaBoostClassifier(n_estimators = 50, learning_rate = 0.6)
sfs3 = RFE(abClf2, n_features_to_select=cutoff)
abX = pd.DataFrame(sfs3.fit_transform(Xtrain_scaled, y_train))
abX.head()